In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time
import pytz
import numpy as np

# ---------------------------------------------------
# TIME RESTRICTION (4 PM – 6 PM IST)
# ---------------------------------------------------
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

if not (time(16, 0) <= current_time <= time(18, 0)):
    print("Stacked Area Chart is available only between 4 PM IST and 6 PM IST.")
else:

    # ---------------------------------------------------
    # LOAD DATA
    # ---------------------------------------------------
    df = pd.read_csv("app_data.csv")

    # ---------------------------------------------------
    # DATA CLEANING
    # ---------------------------------------------------
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce')
    df['installs'] = pd.to_numeric(df['installs'], errors='coerce')
    df['size_mb'] = pd.to_numeric(df['size_mb'], errors='coerce')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    df = df.dropna()

    # ---------------------------------------------------
    # APPLY FILTERS
    # ---------------------------------------------------
    df_filtered = df[
        (df['rating'] >= 4.2) &
        (~df['app_name'].str.contains(r'\d', regex=True)) &
        (df['category'].str.startswith(('T', 'P'))) &
        (df['reviews'] > 1000) &
        (df['size_mb'].between(20, 80))
    ]

    # ---------------------------------------------------
    # TRANSLATE LEGEND LABELS
    # ---------------------------------------------------
    category_translation = {
        "Travel & Local": "Voyage et Local",   # French
        "Productivity": "Productividad",       # Spanish
        "Photography": "写真"                  # Japanese
    }

    df_filtered['category'] = df_filtered['category'].replace(category_translation)

    # ---------------------------------------------------
    # MONTHLY CUMULATIVE INSTALLS
    # ---------------------------------------------------
    df_filtered['month'] = df_filtered['date'].dt.to_period('M')
    monthly = df_filtered.groupby(['month', 'category'])['installs'].sum().reset_index()

    monthly['month'] = monthly['month'].dt.to_timestamp()

    # Pivot table for stacked area chart
    pivot_df = monthly.pivot(index='month', columns='category', values='installs').fillna(0)

    # Sort by month
    pivot_df = pivot_df.sort_index()

    # Cumulative installs
    cumulative_df = pivot_df.cumsum()

    # ---------------------------------------------------
    # CALCULATE MoM GROWTH
    # ---------------------------------------------------
    growth_df = cumulative_df.pct_change()

    # Identify months where any category grew >25%
    highlight_months = growth_df[growth_df > 0.25].dropna(how='all').index

    # ---------------------------------------------------
    # PLOT STACKED AREA CHART
    # ---------------------------------------------------
    plt.figure(figsize=(12, 7))

    x = cumulative_df.index
    y = cumulative_df.T.values

    areas = plt.stackplot(x, y, labels=cumulative_df.columns, alpha=0.7)

    # ---------------------------------------------------
    # HIGHLIGHT SIGNIFICANT GROWTH MONTHS
    # ---------------------------------------------------
    for month in highlight_months:
        plt.axvspan(month, month + pd.offsets.MonthEnd(1), alpha=0.15)

    # ---------------------------------------------------
    # GRAPH SETTINGS
    # ---------------------------------------------------
    plt.title("Cumulative Installs Over Time by Category")
    plt.xlabel("Month")
    plt.ylabel("Cumulative Installs")
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Stacked Area Chart is available only between 4 PM IST and 6 PM IST.
